In [1]:
import os
from glob import glob
import numpy as np
import leroi
import pyart


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/g/data/en0/jss548/miniconda3/envs/openradar/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
#grid config for names and significant digit
GRID_VAR_LIST  = [('DBZH',1),
                    ('VRADH',1)]
#should use dictionaries like val's gridding code - and also set a specific type for every variable

#note: If DP variables are added, leori gridding must be run again within any masking.

GRID_CONFIG  = {'grid_var_list': GRID_VAR_LIST,
                    'GRID_SHAPE': (41, 301, 301),
                    'GRID_LIMITS': ((0, 20000), (-150000.0, 150000.0), (-150000.0, 150000.0)),
                    'GRID_ROI': 2500,
                    'GRID_K': 200,
                    'grid_name': '150km',
                    'description':'Interpolation alogorthim described by Dahl et al. (2019) '
                                'doi:10.1175/JTECH-D-18-0211.1'}

output_folder = '/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset_grid/'

In [9]:
def level1b_grid_worker(radar, grid_config):
    """
    Generated level 1b PPI products

    Parameters:
    ===========
        radar:
            Py-ART radar structure.
        grid_config: dict
            grid configuration dictionary

    Returns:
    ===========
        grid:
            Py-ART grid structure.
        gatefilter:
            Py-ART gatefilter structure.
    """

    if len(radar.fixed_angle["data"]) < 2:
        print('WARNING: GRIDDING ABORTED FOR DUE TO INSUFFICENT SWEEPS')
        return None

    grid_vars, _ = zip(*grid_config['grid_var_list'])

    # check requested fields to grid are present
    var_name_list = []
    for var_name in grid_vars:
        if var_name in radar.fields:
            var_name_list.append(var_name)

    # setup coordinates
    gs = grid_config['GRID_SHAPE']
    gb = grid_config['GRID_LIMITS']
    center_pos = (0, 0, 0)  # radar position on Cartesian grid
    x = np.linspace(gb[2][0], gb[2][1], gs[2])
    y = np.linspace(gb[1][0], gb[1][1], gs[1])
    z = np.linspace(gb[0][0], gb[0][1], gs[0])
    coords = (z - center_pos[0], y - center_pos[1], x - center_pos[2])

    # create grid object
    fields = leroi.leroi_interp(radar, coords, var_name_list, Rc=grid_config['GRID_ROI'], k=grid_config['GRID_K'],
                                verbose=False, multiprocessing=False)

    grid = leroi.build_pyart_grid(radar, fields, gs, gb)

    # ensure grid dtype are always float
    for field_name in grid.fields.keys():
        if field_name != 'radar_echo_classification':
            grid.fields[field_name]['data'] = grid.fields[field_name]['data'].astype(np.single)

    # return grid object
    return grid


In [16]:
odimh5_ffn_list = sorted(glob('/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset/*.h5'))
for odimh5_ffn in odimh5_ffn_list:
    radar = pyart.aux_io.read_odim_h5(odimh5_ffn, file_field_names=True)
    grid = level1b_grid_worker(radar, GRID_CONFIG)
    grid.write(f'{output_folder}/{os.path.basename(odimh5_ffn)[:-3]}_grid.nc', arm_time_variables=True)

In [5]:
#check output
grid = pyart.io.read_grid('/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset_grid/2023072422_24_ODIMH5_PVOL6S_VOL_CASCV_grid.nc')
print(vars(grid))

{'time': {'long_name': 'Time of grid', 'units': 'seconds since 2023-07-24T22:18:07Z', 'standard_name': 'time', 'calendar': 'gregorian', 'data': masked_array(data=[0.],
             mask=False,
       fill_value=1e+20,
            dtype=float32)}, 'fields': {'DBZH': {'_FillValue': -9999.0, 'data': masked_array(
  data=[[[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
  